In [ ]:
import torch

# custom
from AudioCaps.AudioCaps_Dataset import * # 데이터셋
from Clotho.Clotho_Dataset import * # 데이터셋
from transformers import GPT2Tokenizer
from ClipCap_forAAC.CLIPCAP_forAAC import * # network
from Train import *
from util import *

# PANNs를 써먹기 위해 prefix_size를 수정
temporal_prefix_size = 15
global_prefix_size = 11 
prefix_size = temporal_prefix_size + global_prefix_size

transformer_num_layers = {"temporal_num_layers" : 4, "global_num_layers" : 4}
prefix_size_dict = {"temporal_prefix_size" : temporal_prefix_size, "global_prefix_size" : global_prefix_size}


dataset = 'AudioCaps' # 'AudioCaps' or 'Clotho'
tokenizer_type = 'Custom' # 'GPT2' or 'Custom'

TEST_BATCH_SIZE = 5
vocab_size = None

if tokenizer_type == 'Custom' :
    tokenizer = tokenizer_forCustomVocab(Dataset = dataset)
    model_param_name = 'model_AudioCaps_own_vocab.pt'
    vocab_size = len(tokenizer.vocab)
else :
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model_param_name = 'model_Clotho_GPT2_header_freezing.pt'

if dataset == 'AudioCaps' :
    data_dir = './AudioCaps'
    split = 'test'
elif dataset == 'Clotho' :
    data_dir = './Clotho'
    split = 'evaluation'

In [ ]:
test_dataloader = CreateDataloader(tokenizer, data_dir, TEST_BATCH_SIZE, split, prefix_size, is_TrainDataset = False, tokenizer_type = tokenizer_type)

torch.cuda.empty_cache()

USE_CUDA = torch.cuda.is_available() 
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

In [ ]:
model = get_ClipCap_AAC(tokenizer, 
                        vocab_size = vocab_size, Dataset = dataset,
                        prefix_size_dict = prefix_size_dict, transformer_num_layers = transformer_num_layers, 
                        encoder_freeze = False, decoder_freeze = True,
                        pretrain_fromAudioCaps = False, device = device)

model_path = './Trained_model_params/' + model_param_name
model.load_state_dict(torch.load(model_path))

In [ ]:

eval_model(model, test_dataloader, 31, 'test', True)